In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import Ridge
from sklearn.model_selection import KFold, GroupKFold
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from pathlib import Path

# Load processed data
X = pd.read_csv("../data/processed/genotypes_qc_std.csv", index_col=0)

ph = pd.read_csv("../data/processed/phenotypes_clean.csv")
print("Phenotype columns:", ph.columns.tolist())

# Handle case where cultivar is stored as index column
if "cultivar" in ph.columns:
    pheno = ph.set_index("cultivar")
else:
    # typical case: cultivar saved as index
    pheno = ph.set_index(ph.columns[0])
    pheno.index.name = "cultivar"

# Align phenotype with genotypes
pheno = pheno.loc[X.index]

y = pheno["trait_y"].values
groups = pheno["pop"].values

X.shape, y.shape

Phenotype columns: ['index', 'trait_y', 'pop']


((300, 37363), (300,))

In [2]:
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# PCA to capture genetic structure
pca = PCA(n_components=10, random_state=42)
PC = pca.fit_transform(X.values)

# Cluster lines into genetic origins
k = 10  # realistic number of origins/programs
kmeans = KMeans(n_clusters=k, n_init=20, random_state=42)
genetic_group = kmeans.fit_predict(PC)

print("Number of genetic groups:", len(np.unique(genetic_group)))
pd.Series(genetic_group).value_counts().sort_index()

Number of genetic groups: 10


0    46
1    24
2    29
3    20
4    40
5    21
6    25
7    28
8    40
9    27
Name: count, dtype: int64

In [3]:
def cross_validate_ridge(X, y, cv, groups=None, alpha=1.0):
    preds = np.zeros_like(y, dtype=float)

    for train, test in cv.split(X, y, groups):
        model = Ridge(alpha=alpha)
        model.fit(X[train], y[train])
        preds[test] = model.predict(X[test])

    r = pearsonr(y, preds)[0]
    rmse = np.sqrt(mean_squared_error(y, preds))
    return r, rmse

In [4]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)

r_kf, rmse_kf = cross_validate_ridge(
    X.values, y, cv=kf
)

r_kf, rmse_kf


(np.float64(0.09373581724792088), np.float64(1.6532052397033874))

In [5]:
gkf = GroupKFold(n_splits=5)

r_gkf, rmse_gkf = cross_validate_ridge(
    X.values, y, cv=gkf, groups=genetic_group
)

r_gkf, rmse_gkf

(np.float64(0.07673096902700416), np.float64(1.6595842882099388))

In [6]:
results = pd.DataFrame({
    "CV": ["KFold (leakage)", "GroupKFold (genetic groups k=10)"],
    "Predictive_Ability_r": [r_kf, r_gkf],
    "RMSE": [rmse_kf, rmse_gkf]
})

results

,CV,Predictive_Ability_r,RMSE
0,KFold (leakage),0.093736,1.653205
1,GroupKFold (genetic groups k=10),0.076731,1.659584


In [7]:
Path("../outputs/tables").mkdir(parents=True, exist_ok=True)
results.to_csv("../outputs/tables/day3_ridge_cv_results.csv", index=False)

print("Saved: ../outputs/tables/day3_ridge_cv_results.csv")
results

Saved: ../outputs/tables/day3_ridge_cv_results.csv


,CV,Predictive_Ability_r,RMSE
0,KFold (leakage),0.093736,1.653205
1,GroupKFold (genetic groups k=10),0.076731,1.659584
